# Dynamic Programming (Part 2)

In this notebook, we'll implemement dynamic programming algorithms for some more problems: Weighted Independent Set in a Tree and TSP.

##### If you're using Datahub:
* Run the cell below **and restart the kernel if needed**

##### If you're running locally:
* Make sure you've activated the conda environment: `conda activate cs170`
* Launch jupyter: `jupyter notebook` or `jupyter lab` 
* Run the cell below **and restart the kernel if needed**

In [1]:
# Install dependencies
!pip install -r requirements.txt --quiet

In [2]:
import otter
assert (otter.__version__ >= "4.4.1"), "Please reinstall the requirements and restart your kernel."

grader = otter.Notebook("dp2.ipynb")
import pickle
import numpy.random as random
import random
import tqdm
import time
from autograder_utils import is_independent_set, validate_tour, handle_timeout

with open('public_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

rng_seed = 0

## Q1. Weighted Independent Set in a Tree

In class, we saw an algorithm to solve the maximum independent set on trees. For a quick refresher, you can consult https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf#page=21. 

In this problem, we'll add some twists: now, suppose that each vertex $i$ has some weight $w[i]$, and we want to find the independent set with the maximum total weight. **Also, you must implement your algorithm with a top-down approach.**

As before, the algorithm discussed in lecture and the textbook only returns the size of the maximum independent set. However, here we want you to return the list of vertices in the acual independent set. To find the actual independent set, it may be useful to maintain an array seperate from the DP array which can help us backtrack to reconstruct the actual set (simply storing the entire set so far at each entry in the DP table is too expensive and will cause the autograder to fail).

Hint: To compute a node's children and grandchildren, feel free to re-use your code from previous homeworks.

In [33]:
def max_independent_set(adjacency_list, weights):
    """
    Return a list containing the vertices in the maximum weighted independent set.

    args:
        adjacency_list:ListList[[int]] = the adjacency list of the tree. 
        weights:List[int] = a list of vertex weights. weights[i] is the weight of vertex i.

    return:
        List[int] Containing the labels of the vertices in the maximum weighted independent set.
    """
    
#     

#     I = [0]*len(weights)
#     L = [[]]*len(weights)
#     def traversal(tree, n):
#         if tree[n] == []:
#             I[n] = weights[n]
#             L[n] = [n]
#             return [n]
        
#         sumChild, c= [], 0
#         sumGrand, g= [], 0
#         for i in tree[n]:
#             k = traversal(tree, i)
#             if I[i] >= 0:
#                 sumChild += k
#                 c += I[i]
#             for k in tree[i]:
#                 x = traversal(tree, k)
#                 if I[k] >= 0:
#                     sumGrand += x
#                     g += I[k]
#         if c >=  weights[n] + g:
#             I[n] = c
#             L[n] = sumChild
#             return sumChild
#         else:
#             I[n] = weights[n] + g
#             L[n] = sumGrand + [n]
#             return sumGrand + [n]
                
#     print(adjacency_list, weights)
#     traversal(tree, 0)
#     index = I.index(max(I))

#     return L[index]
        

    n = len(adjacency_list)
    
    # Initialize DP arrays
    withN = [0] * len(adjacency_list)  # Maximum weight when including the current vertex
    woN = [0] * len(adjacency_list)  # Maximum weight when excluding the current vertex
    
    # Helper function for the recursive algorithm
    tree = adjacency_list
    
    def construct_tree(p, index):
        """construct a tree using adjacency)list"""
        for i in index:
            if p in tree[i]:
                tree[i].remove(p)
            construct_tree(i, tree[i])

    construct_tree(0, tree[0])
    
    def traversal(node):
        withN[node] = weights[node]
        woN[node] = 0
        for n in tree[node]:
            traversal(n)
            withN[node] += woN[n]
            woN[node] += max(withN[n], woN[n]) 

    traversal(0)
    

    s = []
    
    # Helper function to backtrack and find the independent set
    def create(node, withNode, parent):
        if withNode:
            s.append(node)
        for child in tree[node]:
            if withNode:
                create(child, False, node)
            else:
                if withN[child] <= woN[child]:
                    create(child, False, node)
                else:
                    create(child, True, node)
    
    # Start reconstruction from the root node
    if withN[0] <= woN[0]:
        create(0, False, -1)
    else:
        create(0, True, -1)
    
    return s

### Debugging
A simplified verion of the otter tests are pasted here for your convenience. Feel free to add whatever print statements or assertions you'd like when debugging.

__Note: your solution should not take inordinate amounts of time to run. Our implementation runs on all local test cases in <1s, and your implementation should run in similar time.__

_Points:_ 3

In [34]:
for adj_list, weights, expected_weight in tqdm.tqdm(test_data['q1.2']):
    try:
        result = max_independent_set(adj_list, weights)
    except Exception as e:
        raise Exception(f"Your function threw an error (see above)") from e
    assert is_independent_set(adj_list, result), f"Your output is not an independent set"
    # print(expected_weight)
    assert sum(weights[i] for i in result) == expected_weight, f"Your output is not a maximum independent set"

100%|██████████████████████████████████████████| 30/30 [00:00<00:00, 236.93it/s]


In [16]:
grader.check("independent set")

100%|██████████████████████████████████████████| 30/30 [00:00<00:00, 265.57it/s]


independent set results: All test cases passed!

## Traveling Salesperson DP
Now, we'll implement the dynamic programming algorithm for the traveling salesperson problem (TSP). A brute force solution will be hopelessly slow even for moderate-sized test cases, but we can use dynamic programming to get a solution in slightly more reasonable (but still exponential) time. For a refresher on the TSP algorithm, see Lecture 14 or https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf#page=20. 

As with previous problems, we want you to return the actual tour, not the cost of the tour. We can once again apply the same procedure of backtracking through our subproblem array to reconstruct this tour.

##### A note on implementation:
Since our subproblem definition takes in a set as one of its parameters, we can't just use a 2D array to store our subproblems. Instead, we recommend storing subproblems in a dictionary, where the keys are tuples of the form `(S, i)`, where `i` represents the last city visited before returning home and `S` is the set of cities visited so far. 

To ensure that the keys are hashable, we recommend using Python's built-in `frozenset` class for `S`. `frozenset` is built-in to Python so you can use it without any additional imports, and works just like a normal set, except that it is immutable and hashable. You can read more about `frozenset` here: https://docs.python.org/3/library/stdtypes.html#frozenset.

An alternative approach would be to use a 2D array as usual, but use a bitmask to represent the set of visited cities. In this approach, `S` would be represented as an $n$-bit unsigned integer, and the $i$-th bit of `S` would be set to 1 if and only if the $i$-th city is part of the set of visited cities. Then, since `S` is an integer, we can use it to index into our 2D array.

As with before, storing the entire tour at each step is too memory-intensive and will cause the autograder to fail. Instead, consider maintaining a separate dictionary or array which stores a smaller amount of information but can still help you reconstruct the tour.

In [7]:
import itertools


def tsp_dp(dist_arr):
    """Compute the exact solution to the TSP using dynamic programming and returns the optimal path.

    Args:
        dist_arr (ndarray[int]]): An n x n matrix of distances between cities. dist_arr[i][j] is the distance from city i to city j.

    Returns:
        List[int]: A list of city indices representing the optimal path.
    """
    S = set()
    S.add(1)
    S = frozenset(S)
    C = {}

    nodes = [i + 1 for i in range(len(dist_arr))]
    C[(S, 1)] = 0
    for i in range(2, len(dist_arr) + 1):
        for subS in itertools.combinations(nodes, i):
            if 1 in subS:
                subS = frozenset(subS)
                C[(subS, 1)] = float("inf")
                # print(subS)
                for n in subS:
                    if n != 1:
                        ssS = set(subS) - set([n])
                        # for j in subS:
                        #     if j != n:
                        #         ssS.add(j)
                        ssS = frozenset(ssS)
                        # print(ssS, n)
                        C[(subS, n)] = min([C[(ssS , k)] + dist_arr[k - 1][n - 1] for k in ssS])


    m = -1
    small = float("inf")
    
    for k in nodes:
        if k != 1:
            if small > C[(frozenset(nodes), k)] + dist_arr[k - 1][0]:
                small = C[(frozenset(nodes), k)] + dist_arr[k - 1][0]
                o = (C[(frozenset(nodes), k)] , dist_arr[k - 1][0])
                m = k
    l = [m]
    while len(nodes) != 1:
        s = set(nodes)
        s.remove(m)
        k = C[(frozenset(nodes), m)]
        for i in nodes:
            x = dist_arr[i - 1][m - 1]

            if (frozenset(s), i) in C and k == x + C[(frozenset(s), i)]:
                l.append(i)
                nodes = s
                m = i
                break
                
    # while len(nodes) != 1:
    #     print(nodes)
    #     s = list(nodes)
    #     s.remove(m)
    #     k = C[(frozenset(nodes), m)]
    #     small = float("inf")
    #     for i in nodes:
    #         x = dist_arr[i - 1][m - 1]
    #         if (frozenset(s), i) in C and k == x + C[(frozenset(s), i)]:
    #             if C[(frozenset(s), i)] < small:
    #                 small = C[(frozenset(s), i)]
    #                 l.append(i)
    #                 nodes = s
    #                 m = i
    #                 print(m)
                   

    # l.append(1)
    l = list(reversed([i-1 for i in l]))
    # print(dist_arr)
    # print(C)
    # print("minimun distance = ", small)
    # print(l)
    return l




### Debugging
A simplified verion of the otter tests are pasted here for your convenience. Feel free to add whatever print statements or assertions you'd like when debugging.

_Points:_ 6

In [8]:
# tests on very small cases
for dist_arr, expected_distance in tqdm.tqdm(test_data['q2.1']):
    try:
        result = tsp_dp(dist_arr)
    except Exception as e:
        raise Exception(f"Your function threw an error (see above)") from e
    assert set(result) == set(range(len(dist_arr))), f"Your output does not visit all cities"
    student_length = validate_tour(result, dist_arr)
    assert student_length >= 0, f"Your output is not a valid tour"
    # print("studentd_dis = ", student_length)
    # print("expected_dis = ", expected_distance)
    assert student_length == expected_distance, f"Your output is not a minimum distance tour"

100%|███████████████████████████████████████████| 20/20 [00:06<00:00,  3.02it/s]


In [9]:
grader.check("TSP")

100%|███████████████████████████████████████████| 16/16 [00:06<00:00,  2.60it/s]


TSP results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [ ]:
grader.export(pdf=False, force_save=True)